# OFCOM News Consumption Data - Data Extraction and Creation of Data Structure for Final Cleaning

## 1. Importing the necessary libraries/modules 

In [1]:
import pandas as pd
from functools import reduce
from pathlib import Path
import time
import re

## 2. Getting a list of paths for the different Excel Spreadsheets

In [2]:
path_for_ofcom_datasets = '1_raw_databases/ofcom_covid_data_xlsx'

directory_path = Path(path_for_ofcom_datasets)

file_list = [f.name for f in directory_path.iterdir() if f.is_file()]

file_paths = [f'{path_for_ofcom_datasets}/{file}' for file in file_list if file[-4:] == 'xlsx']


### Reviewing the output - this will be useful so that I can use the paths as an iterable to extract and process the data into its final data structure (uncomment to review)

In [3]:
# file_paths

In [4]:
len(file_paths)

23

## 3. Reviewing the structure of the Excel spreadsheet - determining sheet names & creating a list of all the questions in the OFCOM survey data

### Having reviewed the Excel spreadsheets I am aware the spreadsheets (those of which I have manually reviewed ~ 10) have the same sheet names (INDEX & P1) for each file path I therefore want to confirm this for all 23 sheets

In [5]:
sheet_names = []
all_questions = []

for file_path in file_paths:
    sheet_name = set(pd.ExcelFile(file_path, engine='openpyxl').sheet_names)
    sheet_names.append(sheet_name)
    INDEX = pd.read_excel(file_path, sheet_name=0)
    INDEX.dropna(how='all')
    INDEX.rename(columns={'Unnamed: 2': 'question'}, inplace=True)
    questions = set(INDEX['question'].tolist())
    all_questions.append(questions)

set_sheet_names = reduce(lambda x, y: x.intersection(y), sheet_names)
common_elements = reduce(lambda x, y: x.intersection(y), all_questions)

### Confirming that all the files have sheet Index & P1 and creating a list of the common questions to all 23 spreadsheets and all the questions

In [6]:
set_sheet_names

{'INDEX', 'P1'}

In [7]:
list_questions_common_to_all_sheets = list(common_elements)


In [8]:
all_questions_flattened = set().union(*all_questions)

In [9]:
all_questions_list = list(all_questions_flattened)

In [10]:
len(all_questions_list)

234

## 4. Creating a function which takes a dataframe as an argument and returns the date of the survey (for use later so I can combine dataframes for each question and understand the changes over time)

In [11]:
def determine_date_of_fieldwork(dataframe):
    df = dataframe.copy()
    
    df = df[df.apply(lambda row: row.astype(str).str.contains('ONLINE Fieldwork:', na=False).any(), axis=1)]
    
    filtered_df = df.loc[:, df.apply(lambda col: col.astype(str).str.contains('ONLINE Fieldwork:', na=False).any(), axis=0)]
    
    date = filtered_df.iloc[0].tolist()[0]

    return date

#### Checking the function works:

In [12]:
file_path = file_paths[0]

P1 = pd.read_excel(file_path, sheet_name=1)

file_path = file_paths[0]

INDEX = pd.read_excel(file_path, sheet_name=0)

In [13]:
date = determine_date_of_fieldwork(P1)
date

'ONLINE Fieldwork: 9th to 13th April 2020'

## 5. Reviewing one of the sheet dataframes to see what needs to be done with the data:

In [14]:
P1

,Unnamed: 0,Return to Index,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page 1
1,NaN,NaN,Ofcom Coronavirus Survey - Week 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,ONLINE Fieldwork: 9th to 13th April 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Absolutes/col percents
4,NaN,Table 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5269,NaN,Prepared by Populus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5270,NaN,Note: Sample sizes less than 100 should be tre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### As you can see from the above dataframe there are lots of redundancies - I need to find and separate the tables for each question!

## 6. Splitting the dataframe into individual tables for each question

### I first look through a row to see if there is a cell which contains the string 'Table' and return the row - this function will be used later

In [15]:
def is_table_start(row):
    return row.str.contains(r"Table \d+", na=False).any()

### I then generate a list of dataframes for each question for a given spreadsheet, these will need further cleaning

In [16]:
def generate_uncleaned_tables(dataframe):
    table_start_indices = dataframe.apply(is_table_start, axis=1).index[dataframe.apply(is_table_start, axis=1)].tolist()
    table_start_indices.append(len(dataframe))
    
    tables = []
    
    for i in range(len(table_start_indices) - 1):
        start_idx = table_start_indices[i]
        end_idx = table_start_indices[i + 1]
        table = dataframe.iloc[start_idx:end_idx].reset_index(drop=True)
        tables.append(table)

    return tables

### The length of this list correlates to the number of tables - this means hopefully each table is within its own dataframe although, it will need processing to extract the data and combine it in a meaningful manner to get insights for a region over time

In [17]:
uncleaned_tables = generate_uncleaned_tables(P1)
len(uncleaned_tables)

122

In [18]:
INDEX.describe()

,Ofcom Coronavirus Survey - Week 3,Unnamed: 1,Unnamed: 2,Unnamed: 3
count,122,122,122,122
unique,122,102,97,14
top,Page 1,Table 67,Q12. To what extent do you agree or disagree w...,Base: All respondents who are getting informat...
freq,1,5,6,44


## 7. Cleaning the table

## After much manipulation and reviewing of the output I was able to produce a function which processes each unclean table and does an initial clean

### I first reviewed a dataframe from the initial split for the first question (index 0 in the list) to see what needs to be done and how I can extract the data (uncomment to review)

In [19]:
# uncleaned_tables[0].head

In [20]:
def first_clean_dataframe(i, dataframe, date):
    try:         
        dataframe.drop(columns=dataframe.columns[dataframe.isin(['@removesigrow@']).any()], inplace=True)
        
        dataframe = dataframe.rename(columns = {'Return to Index': 'response'})
        
        table = dataframe['response'].iloc[0]
        question = dataframe['response'].iloc[1]
        
        alternate_left_column_of_data = dataframe['Unnamed: 3']
        
        alternate_left_top_corner = alternate_left_column_of_data.first_valid_index()

        # print("alternate_left_top_corner:", alternate_left_top_corner)
        
        left_column_of_data = dataframe['Unnamed: 2']
        
        left_column_of_data = left_column_of_data[~left_column_of_data.str.contains('Ofcom Coronavirus Survey|ONLINE Fieldwork', case=False, na=False)]
        
        left_top_corner = left_column_of_data.first_valid_index()

        # print("left_top_corner:", left_top_corner)
        
        left_bottom_corner = left_column_of_data[::-1].first_valid_index()

        # print("left_bottom_corner:", left_bottom_corner)

        def first_valid_index_from_right(row):
            valid_indices = row.notna()[::-1]
            index = valid_indices.idxmax() if valid_indices.any() else None
            return index
        
        if alternate_left_top_corner > left_top_corner:
            left_top_corner=alternate_left_top_corner
        
        # print("final left_top_corner:", left_top_corner)

        row = dataframe.iloc[left_top_corner]
        
        right_top_corner = dataframe.columns.get_loc(first_valid_index_from_right(row)) + 1
        # print("right_top_corner:", right_top_corner)

        sliced_dataframe = dataframe.iloc[left_top_corner:left_bottom_corner, :right_top_corner]

        sliced_dataframe.columns = sliced_dataframe.iloc[0]

        sliced_dataframe = sliced_dataframe.dropna(axis=1, how='all')

        sliced_dataframe.rename(columns={ sliced_dataframe.columns[0]: "response" }, inplace = True)
        
        sliced_dataframe = sliced_dataframe.dropna(subset=["response"])
        
        sliced_dataframe = sliced_dataframe[sliced_dataframe['response'] != ' ']
        
        if sliced_dataframe.empty:
            return i

        sliced_dataframe.insert(0, 'date', date)
        sliced_dataframe.insert(0, 'question', question)
        sliced_dataframe.insert(0, 'table', table)
        
        return sliced_dataframe, question
    except:
        print(f"Error in first_clean_dataframe {date}. Table: {i}")
        return f"Error in first_clean_dataframe {date}. Table: {i}", f"Error in first_clean_dataframe {date}. Table: {i}"

## 8. Testing the cleaning function

In [21]:
test_dataframe = first_clean_dataframe(0, uncleaned_tables[0], date)
test_dataframe

(5     table                                 question  \
 6   Table 1  Q1. Device usage to connect to internet   
 7   Table 1  Q1. Device usage to connect to internet   
 8   Table 1  Q1. Device usage to connect to internet   
 11  Table 1  Q1. Device usage to connect to internet   
 15  Table 1  Q1. Device usage to connect to internet   
 19  Table 1  Q1. Device usage to connect to internet   
 22  Table 1  Q1. Device usage to connect to internet   
 25  Table 1  Q1. Device usage to connect to internet   
 29  Table 1  Q1. Device usage to connect to internet   
 33  Table 1  Q1. Device usage to connect to internet   
 37  Table 1  Q1. Device usage to connect to internet   
 40  Table 1  Q1. Device usage to connect to internet   
 
 5                                       date  \
 6   ONLINE Fieldwork: 9th to 13th April 2020   
 7   ONLINE Fieldwork: 9th to 13th April 2020   
 8   ONLINE Fieldwork: 9th to 13th April 2020   
 11  ONLINE Fieldwork: 9th to 13th April 2020   
 15  ONLINE 

### As you can see the cleaning function seems to have created a dataframe which is perfect for further cleaning/manipulation & visualisation

## 8.Using the splitting and cleaning functions for all spreadsheets to obtain all the data
 
### I now want to iterate over all the file paths, split the table for that file into dataframes for individual questions and perform an initial clean on the individual dataframes - this then needs to be stored in a data-structure which will be further processed so that the results of a single dataframe show the results for each question over time

### I now want to iterate over all the file paths, split the table for that file into dataframes for individual questions and perform an initial clean on the individual dataframes - this then needs to be stored in a data-structure which will be further processed so that the results of a single dataframe show the results for each question over time

### Getting a list of dataframes for each Excel file


In [22]:
initial_dataframes_from_file_path = []

for file_path in file_paths:
    dataframe = pd.read_excel(file_path, sheet_name=1)
    initial_dataframes_from_file_path.append(dataframe)

### Determining the dates for each file and saving their value in a list

In [23]:
dates = []

for dataframe in initial_dataframes_from_file_path:
    dates.append(determine_date_of_fieldwork(dataframe))


### Creating an empty data structure of a dictionary of lists which will contain the dataframe relating to each question


In [24]:
cleaned_tables = {question: [] for question in all_questions_list}

In [25]:
tables_split_and_cleaned = 0

for i, initial_dataframe in enumerate(initial_dataframes_from_file_path):
    
    uncleaned_dataframe_for_question = generate_uncleaned_tables(initial_dataframe)

    date = dates[i]

    failed_tables = []

    uncategorized_tables = []
    
    for uncleaned_dataframe in uncleaned_dataframe_for_question:
        initial_clean_dataframe, question = first_clean_dataframe(i, uncleaned_dataframe, date)
        tables_split_and_cleaned += 1
        if initial_clean_dataframe is str:
            failed_tables.append(f"Failed to process Survey date:{date} dataframe: {i}, Question: {question}")
        else:
            if question in cleaned_tables.keys():
                cleaned_tables[question].append(initial_clean_dataframe)
            else:
                uncategorized_tables.append(initial_clean_dataframe)

Error in first_clean_dataframe ONLINE Fieldwork: 4th to 6th December 2020. Table: 17


In [26]:
tables_split_and_cleaned

4232

### The operation was a success for all but one table - this would be sufficient to look at manually if felt necessary!

### There are roughly 200 questions in each spreadsheet and 23 spreadsheets therefore the 4232 tables seems appropriate for the task at hand

## 9. Combining the dataframes within a question

### I first test my code on the first question

In [27]:
question = list_questions_common_to_all_sheets[0]
question

'Q5. Trust in the sources for information/news about Coronavirus used in last week: Websites/apps of online news organisations like Buzzfeed, Huffington Post, Vice, etc.'

#### I had a look at all the dataframes within a list for that question to determine which logic I should use - after playing around with different parameters it seems that some columns are present at certain times and not at others so, I will need to produce a list of columns from all the dataframes, create an empty dataframe with those column names and the add each dataframe to the master dataframe, filling in gaps with Nan

In [28]:
def collect_columns_in_order(list_dataframes):
    all_columns = []
    for dataframe in list_dataframes:
        for column in dataframe.columns:
            if column not in all_columns:
                all_columns.append(column)
    return all_columns

In [29]:
def rename_duplicate_columns(columns):
    seen = {}
    new_columns = []
    
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")  # Rename with suffix
        else:
            seen[col] = 0
            new_columns.append(col)
    
    return new_columns

In [30]:
def combine_dataframes(list_dataframes):
    all_columns = collect_columns_in_order(list_dataframes)
    
    combined_df = pd.DataFrame(columns=all_columns)
    
    for dataframe in list_dataframes:
        dataframe.columns = rename_duplicate_columns(dataframe.columns)
        
        dataframe = dataframe.reindex(columns=all_columns, fill_value=None)
        
        combined_df = pd.concat([combined_df, dataframe], ignore_index=True)

    return combined_df

In [31]:
for question in list_questions_common_to_all_sheets:
    combined_dataframe = combine_dataframes(cleaned_tables[question])

In [34]:
cleaned_combined_tables = {question: [] for question in all_questions_list}

for key in cleaned_combined_tables.keys():
    cleaned_combined_tables[key] = combine_dataframes(cleaned_tables[key])
    time.sleep(0.5)

In [3]:
for key in cleaned_combined_tables.keys():
    cleaned_key = re.sub(r'[^\w\s]', '', str(key))
    cleaned_key = cleaned_key.lower().replace(" ","_")
    cleaned_combined_tables[key].to_csv(f"2_processed_databases/ofcom_data/{cleaned_key}.csv", index=False)

NameError: name 'cleaned_combined_tables' is not defined